# Exercises on working with symbolic expressions

## Exercises 10.1

You may have met the natural algorithm, a special mathematical function written $ ln(x) $. **Draw** the expression $ ln(y \cdot z) $ as a tree built from the elements and combinators described in the previous section.

Using the same template I used in the concepts section, we get to:

![ln(y · z)](../images/ln_y_z.png)


## Exercise 10.2

Translate the expression $ ln(y^z) $ from the previous exercise to Python code. Write it both as a Python function and as a data structure built from elements and combinators.

Hint: the natural logarithm is calculated by the Python function `math.log`.

In [ ]:
Let's start by writing the Python function, followed by the data structure from the classes in `my_expressions.py`:

In [2]:
from my_expressions import *
from math import log

def f(y, z):
    return log(y ** z)

f_expression = Apply(
    Function('ln'),
    Power(
        Variable('y'),
        Variable('z')
    )
)

## Exercise 10.3

What is the expression represented by `Product(Number(3),Sum(Variable("y"),Variable("z")))`?

The previous code represents the expression:

$
\displaystyle
3 \cdot (y + z)
$

Note that the parentheses are not needed in the tree, but required to represent the algebraic expression

## Exercise 10.4

Implement a `Quotient` combinator representing one expression divided by another. How do you represent the following expression?

$
\displaystyle
\frac{a + b}{2}
$

Let's start with the implementation:

In [ ]:
class Quotient():
    def __init__(self, numerator, denominator):
        self.numerator = numerator        
        self.denominator = denominator
        

In [ ]:
Once defined, the previous expression can be easily implemented as:

In [ ]:
from my_expressions import *

expr = Quotient(
    Sum(
        Variable('a'),
        Variable('b')
    ),
    Number(2)
)

## Exercise 10.5

Implement a `Difference` combinator representing one expression subtracted from another. How can you represent the expression $ b^2 - 4ac $?

Let's go straight ahead:

In [ ]:
class Difference():
    def __init__(self, expr1, expr2):
        self.expr1 = expr1
        self.expr2 = expr2

In [ ]:
from my_expressions import *

Difference(
    Power(
        Variable('b'),
        Number(2)
    ),
    Product(
        Number(4),
        Product(
            Variable('a'),
            Variable('c')
        )
    )
)

Note that as `Product` does not allow a variable number of arguments, we had to use function composition to represent $ 4ac $.